In [1]:
import glob
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
from tqdm import tqdm
import h5py

I=0
def _parse_function(example_proto):
    print('1')
    features = {"X": tf.FixedLenFeature((3*257), tf.float32),
              "Y": tf.FixedLenFeature((257), tf.float32)}
    parsed_features = tf.parse_single_example(example_proto, features)
    print("i was here")
    print('2')
    return parsed_features["X"], parsed_features["Y"]

def rbm_layer(n_visible, n_hidden, num_epochs, num_cases, lr, ws, bs, layer_n, len_data, directories):
    Data_path = directories[0]
    tfrecord_folder_parent = directories[1]
    tfrecord_folder = directories[2]
    
    tfrecord_path_x = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_folder))
    sorted_names_x = natsorted(os.listdir(tfrecord_path_x))
    trainfilenames_x = []
    for i in sorted_names_x:
        trainfilenames_x.append(os.path.normpath(os.path.join(tfrecord_path_x,i)))
#     filenames_x = tf.placeholder(tf.string, shape=[None])
#     dataset_x = tf.data.TFRecordDataset(filenames_x)
    dataset_x = tf.data.TFRecordDataset(trainfilenames_x)
    dataset_x = dataset_x.map(_parse_function)  # Parse the record into tensors.
#     dataset_x = dataset_x.repeat()  # Repeat the input indefinitely.
    dataset_x = dataset_x.batch(num_cases)
#     iterator_x = dataset_x.make_initializable_iterator()
    
    weightcost  = 0.0002
    initialmomentum  = 0.5
    finalmomentum    = 0.9
    momentum  = 0.5
    numcases = 32
    W_adder  = tf.zeros((n_visible,n_hidden),dtype=tf.dtypes.float32)
    bh_adder = tf.zeros((1,n_hidden),dtype=tf.dtypes.float32)
    bv_adder = tf.zeros((1,n_visible),dtype=tf.dtypes.float32)
#     x  = tf.placeholder(tf.float32, [None, n_visible], name="x") #The placeholder variable that holds our data
#     h = tf.placeholder(tf.float32, [None, n_hidden], name="h")
#     h = tf.zeros((numcases, n_hidden),dtype=tf.dtypes.float32)
#     h_empty  = np.zeros_like(h)
#     h_empty = np.zeros((, n_hidden),dtype=np.float32)
#     m  = tf.Variable(0.5, dtype=np.float32, name='m')
    W  = tf.Variable(tf.random_normal([n_visible, n_hidden], 0.01), name="W") #The weight matrix that stores the edge weights
    bh = tf.Variable(tf.zeros([1, n_hidden],  tf.float32, name="bh")) #The bias vector for the hidden layer
    bv = tf.Variable(tf.zeros([1, n_visible],  tf.float32, name="bv")) #The bias vector for the visible layer

    def sample(probs):
        #Takes in a vector of probabilities, and returns a random vector of 0s and 1s sampled from the input vector
        return tf.floor(probs + tf.random_uniform(tf.shape(probs), 0, 1))

#     hk = sample(tf.sigmoid(tf.matmul(x, W) + bh)) #Propagate the visible values to sample the hidden values
#     #128*512

#     #Next, we update the values of W, bh, and bv, based on the difference between the samples that we drew and the original values
#     posprods  = tf.matmul(tf.transpose(x),hk) #771*512
#     poshidacts = tf.reduce_sum(hk) #(512)
#     posvisacts = tf.reduce_sum(x,axis=0)#771
    
#     xk   = tf.sigmoid(tf.matmul(hk, tf.transpose(W)) + bv)#128*711
#     neghidprobs   = tf.sigmoid(tf.matmul(xk, W) + bh) #128*512
#     negprods  = tf.matmul(tf.transpose(xk),neghidprobs); #771*512##################################
#     neghidacts = tf.reduce_sum(neghidprobs, axis=0)
#     negvisacts = tf.reduce_sum(xk, axis=0)
#     print(negvisacts)#771
# #     print(type(posprod-negprod))
# #     print(type(lr))
#     m=0.5
#     print('here')
#     W_adder = (m * W_adder)+ (lr*(posprods-negprods)/numcases)-(weightcost*W)
#     bv_adder = (m * bv_adder)+ ((lr/numcases)*(posvisacts-negvisacts))
#     bh_adder = (m * bh_adder)+ ((lr/numcases)*(poshidacts-neghidacts))
#     #When we do sess.run(updt), TensorFlow will run all 3 update steps
#     updt = [W.assign_add(W_adder), bv.assign_add(bv_adder), bh.assign_add(bh_adder)]

    ### Run the graph!
    # Now it's time to start a session and run the graph! 

#     with tf.Session() as sess:
        #First, we train the model
        #initialize the variables of the model
#         sess.run(tf.global_variables_initializer())
#         print(m)
#         sess.run(iterator_x.initializer, feed_dict={filenames_x: trainfilenames_x})
#         print(iterator_x.get_next()[0])
        #Run through all of the training data num_epochs times
#         for epoch in tqdm(range(num_epochs)):
            #Train the RBM on batch_size examples at a time
#             for X_batch in da(batch_size, layer_n, ws, bs, len_data, name, data_name):
#             Data = sess.run(iterator_x.get_next()[0])
#             print('blah')
#             print(Data.shape)
#             for Data in dataset_x:
#                 for j in range(layer_n):
#                     Data = np.matmul(Data,ws[j])+bs[j]
    #             if epoch>5:
    #                 momentum=finalmomentum;
    #             else:
    #                 momentum=initialmomentum;
#                 feed_dict = {x: Data}
#                 var1 = sess.run(updt, feed_dict)
    count=0
    for epoch in tqdm(range(num_epochs)):
        count+=1
        incount=0
        for x in dataset_x:
            incount+=1
            if incount%1000==0:
                print(incount)
            if layer_n>1:
                for j in range(layer_n-1):
                    x = np.matmul(x,ws[j])+bs[j]
            poshidprobs = tf.matmul(x[0], W) + bh #Propagate the visible values to sample the hidden values
            poshidstates = sample(poshidprobs)
            #128*512
            #Next, we update the values of W, bh, and bv, based on the difference between the samples that we drew and the original values
            posprods  = tf.matmul(tf.transpose(x[0]),poshidprobs) #771*512
            poshidacts = tf.reduce_sum(poshidprobs) #(512)
            posvisacts = tf.reduce_sum(x[0],axis=0)#771

            negdata   = tf.sigmoid(tf.matmul(poshidstates, tf.transpose(W)) + bv)#128*711
            neghidprobs   = tf.matmul(negdata, W) + bh #128*512
            negprods  = tf.matmul(tf.transpose(negdata),neghidprobs); #771*512##################################
            neghidacts = tf.reduce_sum(neghidprobs, axis=0)
            negvisacts = tf.reduce_sum(negdata, axis=0)
#             print(negvisacts)#771
        #     print(type(posprod-negprod))
        #     print(type(lr))
            if epoch>5:
                m=finalmomentum
            else:
                m=initialmomentum
            m=0.5
            W_adder = (m * W_adder)+ (lr*(posprods-negprods)/numcases)-(weightcost*W)
            bv_adder = (m * bv_adder)+ ((lr/numcases)*(posvisacts-negvisacts))
            bh_adder = (m * bh_adder)+ ((lr/numcases)*(poshidacts-neghidacts))
            #When we do sess.run(updt), TensorFlow will run all 3 update steps
            updt = [W.assign_add(W_adder), bv.assign_add(bv_adder), bh.assign_add(bh_adder)] 
        print(count)
    return updt




In [2]:
# import libraries.
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
import datetime
tf.enable_eager_execution()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# import keras
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from pystoi.stoi import stoi
import h5py
######################
#import libraries.
import matplotlib.pyplot as plt
from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
####import sounddevice as sd
import time
print('imported')
# #######################
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
# Data_path = os.getcwd()
tfrecord_folder_parent = 'tfrecord_files'
tfrecord_folder = 'tfrecord_files_10h'
tfrecord_val_folder = 'validation_10h'
ckpt_folder = '3'
dirs = [Data_path, tfrecord_folder_parent, tfrecord_folder]
 
# len_data = (684108, 257)
len_data = (2197278, 257)
val_len = (97278,257)
w=3
#######################
#define reconstruct function to reconstruct sound from framed signal.
def reconstruct(wave,angle):
    recon = np.sqrt(np.power(10, wave))
    recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=200, win_length=500, window='hann')
    return recon
#######################
I=0
global batch_size
batch_size = 128
# epochs_num=50
global datalen
datalen=len_data[0]

h = [512,512]
seed = 7
rate1 = 0.1
rate2 = 0.2
from tensorflow.keras.layers import Activation
# from keras.layers import Activation
np.random.seed(seed)
model = Sequential()
act1 = layers.LeakyReLU(alpha=0.1)
model.add(layers.Dropout(rate1, noise_shape=None, seed=None))
# ,kernel_regularizer=regularizers.l2(0.001)
model.add(Dense(h[0], input_dim = w*len_data[1]))
model.add(BatchNormalization())
model.add(act1)
# model.add(Activation('sigmoid'))
act2=layers.LeakyReLU(alpha=0.1)
model.add(layers.Dropout(rate2, noise_shape=None, seed=None))
model.add(Dense(h[1]))
model.add(act2)
# act3=layers.LeakyReLU(alpha=0.1)
# # model.add(layers.Dropout(rate, noise_shape=None, seed=None))
# model.add(Dense(h[2]))
# model.add(act3)
act=layers.LeakyReLU(alpha=0.01)
model.add(Dense(len_data[1]))
#############################################
import os
from natsort import natsorted

# def _parse_function(example_proto):
#     features = {"X": tf.FixedLenFeature((3*257), tf.float32),
#               "Y": tf.FixedLenFeature((257), tf.float32)}
#     parsed_features = tf.parse_single_example(example_proto, features)
#     return parsed_features["X"], parsed_features["Y"]

# tfrecord_path = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_folder))
# sorted_names = natsorted(os.listdir(tfrecord_path))
# trainfilenames = []
# for i in sorted_names:
#     trainfilenames.append(os.path.normpath(os.path.join(tfrecord_path,i)))
# filenames = tf.placeholder(tf.string, shape=[None])
# dataset = tf.data.TFRecordDataset(filenames)
# dataset = dataset.map(_parse_function)  # Parse the record into tensors.
# dataset = dataset.repeat()  # Repeat the input indefinitely.
# dataset = dataset.batch(batch_size)
# iterator = dataset.make_initializable_iterator()

# # orig_path = os.getcwd()
# tfrecord_path_x = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_folder))
# sorted_names_x = natsorted(os.listdir(tfrecord_path_x))
# trainfilenames_x = []
# for i in sorted_names_x:
#     trainfilenames_x.append(os.path.normpath(os.path.join(tfrecord_path,i)))
# filenames_x = tf.placeholder(tf.string, shape=[None])
# dataset_x = tf.data.TFRecordDataset(filenames_x)
# dataset_x = dataset_x.map(_parse_function)  # Parse the record into tensors.
# dataset_x = dataset_x.repeat()  # Repeat the input indefinitely.
# dataset_x = dataset_x.batch(batch_size)
# iterator_x = dataset_x.make_initializable_iterator()

########################
visible = w*len_data[1]
hidden = h[0]
visible1 = h[0]
hidden1 = h[1]
visible2 = h[1]
hidden2 = len_data[1]

layer1 = rbm_layer(visible, hidden, 1, batch_size, 0.01, [np.eye(visible,hidden)], [np.zeros((1,visible))], 1, len_data[0],dirs)
# layer2 = rbm_layer(visible1, hidden1, 50, batch_size, 0.01, [np.eye(visible),layer1[0]], [np.zeros((1,visible)),layer1[2]], 2, len_data[0], dirs)
# layer3 = rbm_layer(visible2, hidden2, 50, batch_size, 0.01, [np.eye(visible),layer1[0],layer2[0]], [np.zeros((1,visible)),layer1[2],layer2[2]], 3, len_data[0], dirs)

###############################

# tfrecord_path_val = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_val_folder))
# sorted_names_val = natsorted(os.listdir(tfrecord_path_val))
# trainfilenames_val = []
# for i in sorted_names_val:
#     trainfilenames_val.append(os.path.normpath(os.path.join(tfrecord_path_val,i)))
# filenames_val = tf.placeholder(tf.string, shape=[None])
# dataset_val = tf.data.TFRecordDataset(filenames_val)
# dataset_val = dataset_val.map(_parse_function)  # Parse the record into tensors.
# dataset_val = dataset_val.repeat()  # Repeat the input indefinitely.
# dataset_val = dataset_val.batch(128)
# iterator_val = dataset_val.make_initializable_iterator()

# epochs_num = 50
# steps = len_data[0] // batch_size
# val_steps = val_len[0] // batch_size
# # You can feed the initializer with the appropriate filenames for the current
# # phase of execution, e.g. training vs. validation.
# # next_elem = iterator_val.get_next()
# # Initialize `iterator` with training data.

# if not os.path.exists(os.path.join(Data_path,"checkpoints",ckpt_folder)):
#     os.makedirs(os.path.join(Data_path,"checkpoints",ckpt_folder))

# print(datetime.datetime.now())
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     sess.run(iterator.initializer, feed_dict={filenames: trainfilenames})
#     sess.run(iterator_val.initializer, feed_dict={filenames_val: trainfilenames_val})
#     print("initialized")
#     checkpoint_path = os.path.normpath(os.path.join(Data_path,"checkpoints",ckpt_folder,"weights.{epoch:02d}.hdf5"))
#     checkpoint_dir = os.path.dirname(checkpoint_path)

#     cp_callback = tf.keras.callbacks.ModelCheckpoint(
#         checkpoint_path, verbose=1, save_best_only=True, save_weights_only=True)
#         # Save weights, every 5-epochs.
# #         period=1)
#     early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
#     opt = tf.keras.optimizers.Adamax()
# #     opt = tf.train.AdamOptimizer()
# #     opt = tf.keras.optimizers.SGD()
#     model.compile(loss='mean_squared_error', optimizer=opt)
#     history = model.fit( iterator, steps_per_epoch=steps,epochs=epochs_num, callbacks = [cp_callback,early_stop], verbose=1,validation_data=iterator_val,validation_steps=val_steps)
# #     model.save(os.path.normpath(os.path.join(Data_path, 'models', "model_3h_dataset.h5")))
# #     tf.keras.models.save_model(model, os.path.normpath(os.path.join(Data_path, 'models', "model_3h_dataset.h5")))
# #     model.save_weights(os.path.normpath(os.path.join(Data_path, 'models', "model_3h_dataset.h5")))
#     model_json = model.to_json()
#     with open(os.path.normpath(os.path.join(Data_path, 'models', "model_3.json")), "w") as json_file:
#         json_file.write(model_json)
#     # # serialize weights to HDF5
#     model.save_weights(os.path.normpath(os.path.join(Data_path, 'models', "model_3.h5")))
#     print("Saved model to disk")
    
# print(datetime.datetime.now())
# %matplotlib inline
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train'], loc='upper left')
# plt.show()
# plt.savefig(os.path.normpath(os.path.join(Data_path,'images',ckpt_folder+'.png')))
# # model_json = model.to_json()
# # with open("model_10h_dataset.json", "w") as json_file:
# #     json_file.write(model_json)
# # model.save_weights("model_10h_dataset.h5")
# # print("Saved model to disk")


imported
1
i was here
2


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Instructions for updating:
Colocations handled automatically by placer.
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
1


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [06:05<00:00, 365.97s/it]


In [3]:
layer1[2]

<tf.Variable 'UnreadVariable' shape=(1, 512) dtype=float32, numpy=
array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [4]:
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/10hdata'
mixed_folder = os.path.normpath(os.path.join(write_path,'ftr_refrmd_10h_norm'))
h5f = h5py.File(mixed_folder+'.hdf5','r')
ftr = h5f['ftr_refrmd_10h_norm'][0:126]
h5f.close()

In [6]:
hidden1 = np.matmul(ftr,layer1[0].numpy())+layer1[2].numpy()
# hidden2 = np.matmul(hidden1,layer2[0])+layer2[2]
# hidden3 = np.matmul(hidden2,layer3[0])+layer3[2]
hidden1_b = np.matmul(hidden1,layer1[0].numpy().T)+layer1[1].numpy()
# hidden2_b = np.matmul(hidden3_b,layer2[0].T)+layer2[1]
# hidden1_b = np.matmul(hidden2_b,layer1[0].T)+layer1[1]

In [7]:
layer1[0]

<tf.Variable 'UnreadVariable' shape=(771, 512) dtype=float32, numpy=
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)>

In [8]:
ftr

array([[-0.24736588, -0.2190181 , -0.15972915, ..., -0.98896515,
        -0.9873588 , -1.473857  ],
       [ 0.3102975 ,  0.1418973 , -0.6608744 , ..., -1.1224244 ,
        -0.9490217 , -0.5731728 ],
       [ 0.14628278,  0.2906629 , -0.39500728, ..., -1.1753767 ,
        -1.5427078 , -1.2799661 ],
       ...,
       [-4.758707  , -5.402432  , -4.871395  , ..., -2.224122  ,
        -2.1833458 , -1.8817462 ],
       [-4.758707  , -5.402432  , -4.871395  , ..., -2.224122  ,
        -2.1833458 , -1.8817462 ],
       [-4.758707  , -5.402432  , -4.871395  , ...,  1.9783895 ,
         1.9585117 ,  1.9666079 ]], dtype=float32)

In [9]:
hidden1_b

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [27]:
a=tf.constant([1,2,3])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    label_numpy = a.eval()

In [97]:
import h5py 
import tensorflow as tf
hh = h5py.File('ftr_refrmd_10h.hdf5', 'r')
d=hh['ftr_refrmd_10h'][0]
len_data=d.shape
hh.close()
len_data

(771,)

In [24]:
file.close()

In [92]:
dataset

<PrefetchDataset shapes: ((?, 257), (?,)), types: (tf.float32, tf.float32)>